In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import glob
import ipywidgets as widgets

# Time series

In [ ]:
runs = glob.glob(
    "/Users/caspar/Developer/mscthesis/data/cases/bomex/output-gcc/run*/tmser.001.nc"
)
ensemble = xr.open_mfdataset(runs, combine="nested", concat_dim="run")

nvidia = xr.open_dataset(
    "/Users/caspar/Developer/mscthesis/data/cases/bomex/output-nvidia/tmser.001.nc"
)

In [ ]:
variable_menu = widgets.Dropdown(
    options=list(nvidia.keys()),
    description="Variable"
)

ui = widgets.HBox([variable_menu])

def update(variable: str) -> None:
    mean = ensemble[variable].mean(dim="run")
    var = ensemble[variable].var(dim="run")
    
    fix, ax = plt.subplots()
    mean.plot.line(x="time", ax=ax)
    ax.fill_between(
        x=var["time"], y1=(2 * np.sqrt(var)) + mean, y2=(-2 * np.sqrt(var)) + mean
    )
    nvidia[variable].plot(x="time", ax=ax)
    
out = widgets.interactive_output(update, {'variable': variable_menu})

display(ui, out)

# Profiles

In [ ]:
runs = glob.glob(
    "/Users/caspar/Developer/mscthesis/data/cases/bomex/output-gcc/run*/profiles.001.nc"
)
ensemble = xr.open_mfdataset(runs, combine="nested", concat_dim="run")

nvidia = xr.open_dataset(
    "/Users/caspar/Developer/mscthesis/data/cases/bomex/output-nvidia/profiles.001.nc"
)

In [ ]:
variable_menu = widgets.Dropdown(
    options=list(nvidia.keys()),
    description="Variable"
)

timestamps_menu = widgets.IntSlider(
    value=0,
    min=0,
    max=len(ensemble.time),
    step=1,
    description="Timestamp",
    continuous_update=False,
    orientation="horizontal"
)

ui = widgets.HBox([variable_menu, timestamps_menu])

def update(variable: str, time: int) -> None:
    mean = ensemble[variable].isel(time=time).mean(dim="run")
    var = ensemble[variable].isel(time=time).var(dim="run")
    
    fix, ax = plt.subplots()
    try:
        mean.plot.line(y="zt", ax=ax, label="GCC ensemble")
        ax.fill_betweenx(
            y=var["zt"], x1=(2 * np.sqrt(var)) + mean, x2=(-2 * np.sqrt(var)) + mean
        )
        nvidia[variable].isel(time=time).plot.line(y="zt", ax=ax, label="OpenACC")
    except:
        mean.plot.line(y="zm", ax=ax, label="GCC ensemble")
        ax.fill_betweenx(
            y=var["zm"], x1=(2 * np.sqrt(var)) + mean, x2=(-2 * np.sqrt(var)) + mean
        )
        nvidia[variable].isel(time=time).plot.line(y="zm", ax=ax, label="OpenACC")
    ax.legend()

out = widgets.interactive_output(update, {'variable': variable_menu, 'time': timestamps_menu})

display(ui, out)